# split by innerBC

In [1]:
import gzip, time
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import seaborn as sns
from Bio.Seq import Seq
from collections import Counter
import os

In [2]:
sample_paths = {
    'a3028_r1': {
        'fastq_paths':['/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz']
    },
    'a3028_r2': {
        'fastq_paths':['/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz']
    },
    'a3028_r3': {
        'fastq_paths':['/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ24003922-YJH-cas9-50-YJH-cas9-50/SQ24003922-YJH-cas9-50-YJH-cas9-50_combined_{}.fastq.gz']
    }
}

In [3]:
def get_innerBC(r2_line):
    innerBC = r2_line[0:6]
    num_Ns = sum([c=='N' for c in innerBC])
    innerBC_ref = ['AATCCG','AATCGC','AAGTCG','AAGCTC','AACGTG','AACTGC','ATAGCG',
                  'ATTCCG','ATGCCA','ATGTTC','ATCACG','ATCCAG','ACAGTG','ACTCTG','ACTTGA',
                  'ACGATC'] # target gene sequencing index
    if num_Ns > 1: 
        return None 
    elif num_Ns == 1 and innerBC == 'ANTCCG':
        return None # will miss some reads
    elif num_Ns == 1 and np.any([innerBC.replace('N',c) in innerBC_ref for c in 'ACTG']): 
        return [innerBC.replace('N',c) for c in 'ACTG' if innerBC.replace('N',c) in innerBC_ref ][0]
    elif innerBC in innerBC_ref: 
        return innerBC
    else: 
        return None
    
def hamming(bc1,bc2): return np.sum([x1 != x2 for x1,x2 in zip(bc1,bc2)])

In [4]:
innerBC_reads_count = {}
innerBC_reads_count['r1'] = {}
innerBC_reads_count['r2'] = {}

for sample,paths in sample_paths.items():
    for fastq_path in paths['fastq_paths']:
        R1 = gzip.open(fastq_path.format('R1'))
        R2 = gzip.open(fastq_path.format('R2'))
        counter = 0
        start_time = time.time()
        while True:
            counter += 1
            if counter % 1000000 == 0: print(fastq_path+ ': Processed {} reads in {} seconds'.format(counter, time.time()-start_time))
            try:
                r1_line = R1.readline().decode('utf-8')
                r2_line = R2.readline().decode('utf-8')
            except:
                print('ERROR extracting {}'.format(fastq_path))
                break
            if r2_line == '' : break
            if r2_line[0] == '@': 
                read2_name = r2_line
                read1_name = r1_line
                
                read1_seq = R1.readline().decode('utf-8')
                read2_seq = R2.readline().decode('utf-8')
                
                if get_innerBC(read2_seq)!=None:
                    innerBC = get_innerBC(read2_seq)
                    read2_QI = R2.readline().decode('utf-8')
                    read2_baseQ = R2.readline().decode('utf-8')
                    read2 = [read2_name , read2_seq , read2_QI , read2_baseQ]

                    read1_QI = R1.readline().decode('utf-8')
                    read1_baseQ = R1.readline().decode('utf-8')
                    read1 = [read1_name , read1_seq , read1_QI , read1_baseQ]

                    #print(read1)
                    
                    if innerBC in innerBC_reads_count['r1']:
                        innerBC_reads_count['r1'][innerBC] += read1
                        innerBC_reads_count['r2'][innerBC] += read2
                    else:
                        innerBC_reads_count['r1'][innerBC] = read1
                        innerBC_reads_count['r2'][innerBC] = read2
                else:
                    read2_QI = R2.readline().decode('utf-8')
                    read2_baseQ = R2.readline().decode('utf-8')
                    read2 = [read2_name , read2_seq , read2_QI , read2_baseQ]
                    
                    read1_QI = R1.readline().decode('utf-8')
                    read1_baseQ = R1.readline().decode('utf-8')
                    read1 = [read1_name , read1_seq , read1_QI , read1_baseQ]

                    if 'unclassified' in innerBC_reads_count['r1']:
                        innerBC_reads_count['r1']['unclassified'] += read1
                        innerBC_reads_count['r2']['unclassified'] += read2
                    else:
                        innerBC_reads_count['r1']['unclassified'] = read1
                        innerBC_reads_count['r2']['unclassified'] = read2
                #break     

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 1000000 reads in 10.713467836380005 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 2000000 reads in 21.15733528137207 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 3000000 reads in 31.63232183456421 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 4000000 reads in 42.12303304672241 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 40000000 reads in 422.00494360923767 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 41000000 reads in 432.7706456184387 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 42000000 reads in 443.76592993736267 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 43000000 reads in 454.5590589046478 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_comb

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 79000000 reads in 845.4850544929504 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ23040124-YJH-cas9-14-YJH-cas9-14/SQ23040124-YJH-cas9-14-YJH-cas9-14_combined_{}.fastq.gz: Processed 80000000 reads in 856.264924287796 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz: Processed 1000000 reads in 10.939930438995361 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz: Processed 2000000 reads in 21.955224990844727 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz: Processed 31000000 reads in 339.9524166584015 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz: Processed 32000000 reads in 350.7847716808319 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/s2164g01077_MCS-20231230-L-04-2024-01-021720/Sample_SQ23071627-YJH-cas9-36-YJH-cas9-36/SQ23071627-YJH-cas9-36-YJH-cas9-36_combined_{}.fastq.gz: Processed 33000000 reads in 361.53824281692505 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/a3028/Sample_SQ24003922-YJH-cas9-50-YJH-cas9-50/SQ24003922-YJH-cas9-50-YJH-cas9-50_combined_{}.fastq.gz: Processed 1000000 reads in 10.631780862808228 seconds
/syn1/li

In [5]:
os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1/a3028/fastq/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1/a3028/fastq'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)

In [6]:
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1/a3028/fastq/*.fq|xargs -I {} gzip {}')

0

In [7]:
print('finished')

finished
